# Greeks computation using different methods

In [2]:
import numpy as np
from scipy.stats import norm

## Bump methods for Delta

In [3]:
def bs_price(S, K, r, sigma, T, is_call=True) -> float :
    """
    Prix Black-Scholes d'une option européenne.
    """
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    
    if is_call:
        return S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    else:
        return K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)

In [4]:

def delta_bump(S, K : float, r : float, sigma : float, T ,is_call=True, h=None):
    """
    Calcule le Delta via la méthode du bump (différences finies centrées).
    """
    # Taille du bump (par défaut : 1% du sous-jacent)
    if h is None:
        h = 0.01 * S
    
    M = int(4/(h**4))
    print(M)
    sum_derivatives = 0
    for i in range(M) :
        # Prix avec S+h et S-h
        V_up = bs_price(S + h, K, r, sigma, T, is_call)
        V_down = bs_price(S - h, K, r, sigma, T, is_call)
        
        # Approximation de la dérivée
        delta = (V_up - V_down) / (2 * h)
        sum_derivatives+=delta
    
    Delta = sum_derivatives/M
    
    return Delta

In [5]:
S = 100      # spot
K = 100      # strike
r = 0.05     # taux sans risque
sigma = 0.2  # volatilité
T = 1        # maturité (en années)

delta_estime = delta_bump(S, K, r, sigma, T, is_call=True, h=0.1)
print("Delta (méthode bump):", delta_estime)

39999
Delta (méthode bump): 0.6368297912531728


In [6]:
d1 = (np.log(S/K) + (r + 0.5*sigma**2)*T) / (sigma*np.sqrt(T))
delta_theorique = norm.cdf(d1)  # pour un call
print("Delta (formule analytique):", delta_theorique)

Delta (formule analytique): 0.6368306511756191


## Pathwise method for Delta

In [7]:
def Delta_pathwise(S, K, T, r, sigma, N):
    Z = np.random.normal(0,1,N)
    S_PT = S*np.exp((r-0.5*sigma**2)*T*np.ones((N, 1))+sigma*np.sqrt(T)*Z)
    payoffs = S_PT*(S_PT>K)/S
    delta = np.exp(-r*T)*np.mean(payoffs)

    return delta

Delta_pathwise(S,K,T,r,sigma,10000)

np.float64(0.6393393538464519)

## Computation of the Delta of an Asian Call

In [8]:
def asian_call_delta_pathwise(S0, K, r, sigma, T, M=50, N=100_000, seed=None):

    if seed is not None:
        np.random.seed(seed)

    dt = T / M
    drift = (r - 0.5 * sigma**2) * dt
    diffusion = sigma * np.sqrt(dt)

    # Simulate Brownian increments
    Z = np.random.randn(N, M)
    S = np.zeros((N, M + 1))
    S[:, 0] = S0

    # Simulate paths
    for i in range(1, M + 1):
        S[:, i] = S[:, i-1] * np.exp(drift + diffusion * Z[:, i-1])

    # Arithmetic average of each path
    S_avg = S[:, 1:].mean(axis=1)

    # Payoff indicator
    indicator = (S_avg > K).astype(float)

    # Pathwise derivative term (mean(S_t) / S0)
    dS_dS0 = S[:, 1:].mean(axis=1) / S0

    # Discounted expectation
    delta = np.exp(-r * T) * np.mean(indicator * dS_dS0)

    return delta

S0 = 100
K = 100
r = 0.05
sigma = 0.2
T = 1.0

delta_est = asian_call_delta_pathwise(S0, K, r, sigma, T, M=50, N=1_000_000, seed=42)
print(f"Pathwise Delta ≈ {delta_est:.6f}")

Pathwise Delta ≈ 0.590239


In [9]:
import numpy as np

def asian_call_vega_pathwise(S0, K, r, sigma, T, M=50, N=100_000, seed=None, antithetic=False):

    if seed is not None:
        np.random.seed(seed)

    dt = T / M
    sqrt_dt = np.sqrt(dt)
    times = np.arange(1, M+1) * dt  # t1, t2, ..., tM shape (M,)

    # If antithetic, we'll generate N/2 normals and reflect them
    if antithetic:
        if N % 2 == 1:
            N += 1  # make even
        half = N // 2
        Z_half = np.random.randn(half, M)
        Z = np.vstack([Z_half, -Z_half])
    else:
        Z = np.random.randn(N, M)

    # Build Brownian paths W_{t_i} = sqrt(dt) * cumsum(Z, axis=1)
    W = np.cumsum(Z, axis=1) * sqrt_dt  # shape (N, M)

    # simulate asset paths: S_{t_i} for i=1..M
    drift_increment = (r - 0.5 * sigma**2) * dt
    # compute exponent increments and cumulatively multiply
    # We can compute S directly stepwise (vectorized)
    S = np.empty((N, M+1))
    S[:, 0] = S0
    for i in range(1, M+1):
        S[:, i] = S[:, i-1] * np.exp(drift_increment + sigma * Z[:, i-1])

    S_t = S[:, 1:]  # shape (N, M) for S_{t1}..S_{tM}

    # arithmetic average
    S_avg = S_t.mean(axis=1)  # shape (N,)

    # payoff indicator for call
    indicator = (S_avg > K).astype(float)

    # compute partial derivative dS_t / dσ for each t_i:
    # dS_ti_dsigma = S_ti * (-sigma * t_i + W_ti)
    # times shape (M,), W shape (N, M) -> broadcast
    dS_dsigma_t = S_t * ( -sigma * times[np.newaxis, :] + W )  # shape (N, M)

    # average to get dSbar/dsigma
    dSbar_dsigma = dS_dsigma_t.mean(axis=1)  # shape (N,)

    # discounted expectation
    pathwise_term = indicator * dSbar_dsigma
    discounted = np.exp(-r * T) * pathwise_term

    vega = discounted.mean()
    stderr = discounted.std(ddof=1) / np.sqrt(N)

    return vega, stderr


In [10]:
S0 = 100.0
K = 100.0
r = 0.03
sigma = 0.2
T = 1.0
M = 50
N = 200_000

vega_est, se = asian_call_vega_pathwise(S0, K, r, sigma, T, M=M, N=N, seed=123, antithetic=True)
print(f"Vega ≈ {vega_est:.6f}, stderr ≈ {se:.6f}")

Vega ≈ 131.357213, stderr ≈ 1.136610


## Log Likelihood Ratio method

In [12]:
def Delta_Asian_call_MC_BS_LLR(r,S0,sigma,T,K,N,n):
    
    delta=float(T/n)

    G=np.random.normal(0,1,size=(N,n))

    #Log returns
    LR=(r-0.5*sigma**2)*delta+np.sqrt(delta)*sigma*G
    # concatenate with log(S0)
    LR=np.concatenate((np.log(S0)*np.ones((N,1)),LR),axis=1)
    # cumsum horizontally (axis=1)
    LR=np.cumsum(LR,axis=1)
    # take the expo Spath matrix
    Spaths=np.exp(LR)
    #print(Spaths)
    
    ## Discrete Asian option over n future dates (St1+St2+....+Stn/n)
    #remove first time component
    Spaths=Spaths[:,1:len(Spaths[0,:])]
    #print(Spaths)
    
    #take the average over each row
    Sbar=np.mean(Spaths,axis=1)
   
    Derivative_Weight=G[:,0]/(S0*sigma*np.sqrt(delta))
    
    # Delta payoff by LLR method
    Delta_payoff_LLR=np.exp(-r*T)*np.maximum(Sbar-K,0)*Derivative_Weight 
    
    Delta_Asian_MC_price_LLR=np.mean(Delta_payoff_LLR)

    # 95% C.I

    STD=np.std(Delta_payoff_LLR) # standard deviation estimator

    error=1.96*STD/np.sqrt(N)

    CI_up=Delta_Asian_MC_price_LLR + error
    CI_down=Delta_Asian_MC_price_LLR -error
    
    return Delta_Asian_MC_price_LLR,CI_up,CI_down,error




[Delta_Asian_MC_price_LLR,CI_up,CI_down,error]=Delta_Asian_call_MC_BS_LLR(0.05,100,0.2,1,100,100000,10)    
print('*******Delta Asian CAll LLR *****')
print(Delta_Asian_MC_price_LLR,CI_up,CI_down,error)

*******Delta Asian CAll LLR *****
0.6008763412052478 0.6132685463344315 0.5884841360760641 0.012392205129183754


In [41]:
def Vega_Asian_Call_LLR(S0, K, T, r, sigma, N, M):
    dt = T/M
    
    G=np.random.normal(0,1,size=(N,M))

    volatility_term = sigma*np.sqrt(dt)
    drift = (r - 0.5*sigma**2)*dt
    
    S_t = S0*np.exp(np.cumsum(drift + volatility_term*G, axis=1))
    
    S_bar = np.mean(S_t, axis=1)

    payoff = np.maximum(S_bar - K, 0)

    derivative_weight = np.sum(G*np.sqrt(dt) - sigma * dt, axis=1)

    vega = np.exp(-r*T)*np.mean(payoff*derivative_weight)

    STD = np.std(np.exp(-r*T)*payoff*derivative_weight)

    CI_up = vega + 1.96*STD/np.sqrt(M)
    print('CI up :', CI_up)
    CI_down = vega - 1.96*STD/np.sqrt(M)
    print('CI down :', CI_down)

    return vega


print(Vega_Asian_Call_LLR(100,110,1,0.5,0.2,10,100000))
    

CI up : 11.228641888468413
CI down : 11.037719421159427
11.13318065481392
